# CQF Final Project: Blending Ensemble for Classification

**Topic**: ML - Blending Ensemble for Classification  
**Asset**: SPY ETF  
**Prediction**: 5-day forward return direction (binary classification)  
**Author**: LIU JingZhong Eric  
**Date**: 2025

---

## Blending Ensemble Model

- **Blending**: Split training data into base_train (70%) and blend_holdout (30%)
  - Base learners trained on base_train
  - Base learners predict on blend_holdout to create meta-features
  - Meta-learner trained on blend_holdout meta-features
  

This implementation follows the **BLENDING** approach.

## 1. Environment Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    precision_recall_curve, ConfusionMatrixDisplay, roc_curve, 
    auc, matthews_corrcoef, balanced_accuracy_score, f1_score,
    brier_score_loss, average_precision_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight
from sklearn.base import clone
from scipy.stats import pearsonr, randint as sp_randint, uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.calibration import calibration_curve
import os
import matplotlib.ticker as mtick
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


## 2. Utility Functions

### 2.1 Random Seed Control

In [ ]:
# ==========================================
# 0. RANDOM SEED CONTROL
# ==========================================
def set_seeds(seed=42):
    """Set random seeds for reproducibility"""
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


### 2.2 Data Loading

In [ ]:
# ==========================================
# 1. DATA LOADING AND MERGING
# ==========================================
def load_and_merge(spy_train, spy_test, gdp_file, dgs10_file):
    """
    Load SPY price data and merge with macroeconomic indicators.
    """
    print("=" * 60)
    print("STEP 1: Loading Data")
    print("=" * 60)
    
    df_train_raw = pd.read_csv(spy_train, parse_dates=['Date'], index_col='Date').sort_index()
    df_test_raw = pd.read_csv(spy_test, parse_dates=['Date'], index_col='Date').sort_index()
    
    oos_start_date = df_test_raw.index.min()
    print(f"  Training period: {df_train_raw.index.min()} to {df_train_raw.index.max()}")
    print(f"  Test period: {df_test_raw.index.min()} to {df_test_raw.index.max()}")
    
    df = pd.concat([df_train_raw, df_test_raw]).sort_index()
    
    # Clean price columns
    for col in ['Close/Last', 'Open', 'High', 'Low']:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].str.replace('$', '').astype(float)
    
    if 'Close/Last' in df.columns:
        df.rename(columns={'Close/Last': 'Close'}, inplace=True)
    
    # Load macroeconomic data
    gdp = pd.read_csv(gdp_file, parse_dates=['observation_date'])
    gdp.columns = ['Date', 'GDPC1']
    gdp.set_index('Date', inplace=True)
    
    dgs10 = pd.read_csv(dgs10_file, parse_dates=['observation_date'])
    dgs10.columns = ['Date', 'DGS10']
    dgs10['DGS10'] = pd.to_numeric(dgs10['DGS10'], errors='coerce')
    dgs10.set_index('Date', inplace=True)
    
    df = df.join(gdp.reindex(df.index, method='ffill'))
    df = df.join(dgs10.reindex(df.index, method='ffill'))
    df = df.ffill().bfill()
    
    print(f"  Total samples: {len(df)}")
    print(f"  Date range: {df.index.min()} to {df.index.max()}")
    
    return df, oos_start_date

### 2.3 Fractional Differentiation

In [ ]:
# ==========================================
# 2. FRACTIONAL DIFFERENTIATION
# ==========================================
def get_weights_ffd(d, thres, lim):
    """Calculate weights for fractional differentiation"""
    w, k = [1.], 1
    while True:
        w_k = -w[-1] / k * (d - k + 1)
        if abs(w_k) < thres:
            break
        w.append(w_k)
        k += 1
        if k >= lim:
            break
    return np.array(w[::-1]).reshape(-1, 1)

def frac_diff_ffd(series, d=0.4, thres=1e-5):
    """Apply fractional differentiation to make series stationary while preserving memory."""
    w = get_weights_ffd(d, thres, len(series))
    width = len(w) - 1
    df = {}
    for name in series.columns:
        series_f = series[[name]].ffill().dropna()
        df_temp = pd.Series(0.0, index=series_f.index)
        for k in range(width, len(series_f)):
            df0 = series_f.iloc[k-width:k+1].values
            df_temp.iloc[k] = np.dot(w.T, df0)[0, 0]
        df[name] = df_temp.loc[series_f.index[width]:]
    return pd.DataFrame(df)

## 3. Feature Engineering

In [ ]:
# ==========================================
# 3. COMPREHENSIVE FEATURE ENGINEERING
# ==========================================
def create_features_enhanced(df):
    """
    Create 40+ features across multiple categories.
    All features use LAGGED data to prevent look-ahead bias.
    """
    print("\n" + "=" * 60)
    print("STEP 2: Feature Engineering")
    print("=" * 60)
    
    df = df.copy()
    
    # Create lagged price/volume
    df['prev_close'] = df['Close'].shift(1)
    df['prev_open'] = df['Open'].shift(1)
    df['prev_high'] = df['High'].shift(1)
    df['prev_low'] = df['Low'].shift(1)
    df['prev_volume'] = df['Volume'].shift(1)
    
    # ========== 1. MOMENTUM INDICATORS ==========
    df['returns_1d'] = df['prev_close'].pct_change(1)
    df['returns_5d'] = df['prev_close'].pct_change(5)
    df['returns_10d'] = df['prev_close'].pct_change(10)
    df['returns_20d'] = df['prev_close'].pct_change(20)
    df['momentum_5d'] = df['prev_close'] - df['Close'].shift(6)
    df['momentum_10d'] = df['prev_close'] - df['Close'].shift(11)
    df['acceleration'] = df['returns_1d'] - df['returns_1d'].shift(1)
    df['ROC_5'] = ((df['prev_close'] - df['Close'].shift(6)) / (df['Close'].shift(6) + 1e-8)) * 100
    df['ROC_10'] = ((df['prev_close'] - df['Close'].shift(11)) / (df['Close'].shift(11) + 1e-8)) * 100
    
    # ========== 2. TREND INDICATORS ==========
    for window in [5, 10, 20, 50]:
        ma = df['prev_close'].rolling(window).mean()
        df[f'MA_{window}'] = ma
        df[f'MA_{window}_ratio'] = df['prev_close'] / (ma + 1e-8)
    
    ema_fast = df['prev_close'].ewm(span=5, adjust=False).mean()
    ema_slow = df['prev_close'].ewm(span=20, adjust=False).mean()
    df['trend_strength'] = (ema_fast - ema_slow) / (ema_slow + 1e-8)
    
    exp1 = df['prev_close'].ewm(span=12, adjust=False).mean()
    exp2 = df['prev_close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    # ========== 3. VOLATILITY INDICATORS ==========
    df['volatility_5'] = df['returns_1d'].rolling(5).std()
    df['volatility_10'] = df['returns_1d'].rolling(10).std()
    df['Vol_20'] = df['returns_1d'].rolling(20).std()
    df['volatility_ratio'] = df['volatility_5'] / (df['Vol_20'] + 1e-8)
    
    roll_mean_20 = df['prev_close'].rolling(20).mean()
    roll_std_20 = df['prev_close'].rolling(20).std()
    df['bb_position'] = (df['prev_close'] - roll_mean_20) / (2 * roll_std_20 + 1e-8)
    df['bb_width'] = (4 * roll_std_20) / (roll_mean_20 + 1e-8)
    
    # ATR
    high_low = df['prev_high'] - df['prev_low']
    high_close = np.abs(df['prev_high'] - df['prev_close'].shift(1))
    low_close = np.abs(df['prev_low'] - df['prev_close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(14).mean()
    
    # ========== 4. VOLUME INDICATORS ==========
    vol_ma_10 = df['prev_volume'].rolling(10).mean()
    df['volume_ratio'] = df['prev_volume'] / (vol_ma_10 + 1e-8)
    df['volume_momentum'] = df['prev_volume'].pct_change(5)
    
    hl_range = df['prev_high'] - df['prev_low']
    df['money_flow'] = ((2 * df['prev_close'] - df['prev_high'] - df['prev_low']) / (hl_range + 1e-8)) * df['prev_volume']
    
    # ========== 5. CANDLESTICK PATTERNS ==========
    df['intraday_momentum'] = (df['prev_close'] - df['prev_open']) / (df['prev_high'] - df['prev_low'] + 1e-8)
    df['candle_body_ratio'] = (df['prev_close'] - df['prev_open']).abs() / (df['prev_high'] - df['prev_low'] + 1e-8)
    
    # ========== 6. TECHNICAL INDICATORS ==========
    # RSI
    delta = df['prev_close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-8)
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Stochastic
    low_14 = df['prev_low'].rolling(14).min()
    high_14 = df['prev_high'].rolling(14).max()
    df['stoch_k'] = 100 * (df['prev_close'] - low_14) / (high_14 - low_14 + 1e-8)
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    
    # Williams %R
    df['williams_r'] = -100 * (high_14 - df['prev_close']) / (high_14 - low_14 + 1e-8)
    
    # ========== 7. MACROECONOMIC ==========
    df['prev_dgs10'] = df['DGS10'].shift(1)
    df['prev_gdpc1'] = df['GDPC1'].shift(1)
    df['dgs10_change'] = df['prev_dgs10'].diff()
    df['gdpc1_growth'] = df['prev_gdpc1'].pct_change(60)
    
    # ========== 8. FRACTIONAL DIFFERENTIATION ==========
    try:
        frac_df = frac_diff_ffd(df[['prev_close']], d=0.4)
        df['close_frac'] = frac_df['prev_close']
    except:
        df['close_frac'] = df['prev_close'].diff()
    
    feature_count = len([c for c in df.columns if c not in ['Close', 'Open', 'High', 'Low', 'Volume', 'GDPC1', 'DGS10']])
    print(f"  Created {feature_count} features")
    
    return df.dropna()

## 4. Label Definition

In [ ]:
# ==========================================
# 4. LABEL DEFINITION
# ==========================================
def make_labels_momentum(df, lookahead=5, vol_factor=0.5):
    """Create binary labels for classification."""
    print("\n" + "=" * 60)
    print("STEP 3: Label Definition")
    print("=" * 60)
    
    df = df.copy()
    df['future_ret'] = df['Close'].pct_change(lookahead).shift(-lookahead)
    
    if 'Vol_20' not in df.columns:
        df['Vol_20'] = df['Close'].pct_change(1).rolling(20).std()
    
    dynamic_threshold = df['Vol_20'] * vol_factor
    df['Target'] = np.where(df['future_ret'] > dynamic_threshold, 1, 0)
    df['trade_ret'] = df['future_ret']
    
    df = df.dropna(subset=['Target', 'trade_ret'])
    
    class_dist = df['Target'].value_counts(normalize=True) * 100
    print(f"  Lookahead period: {lookahead} days")
    print(f"  Volatility factor: {vol_factor}")
    print(f"  Class distribution:")
    print(f"    Class 0 (Down/Neutral): {class_dist.get(0, 0):.2f}%")
    print(f"    Class 1 (Up): {class_dist.get(1, 0):.2f}%")
    
    return df

## 5. Feature Selection

### 5.1 SOM-based Feature Selection

In [ ]:
# ==========================================
# 5. SOM FOR FEATURE SELECTION
# ==========================================
class SimpleSOM:
    """Simple Self-Organizing Map for feature clustering."""
    def __init__(self, x, y, input_len, learning_rate=0.5, sigma=1.0):
        self.x = x
        self.y = y
        self.learning_rate = learning_rate
        self.sigma = sigma
        self.weights = np.random.random((x, y, input_len))
    
    def _find_bmu(self, sample):
        dists = np.linalg.norm(self.weights - sample, axis=2)
        return np.unravel_index(np.argmin(dists), dists.shape)
    
    def fit(self, data, epochs=100):
        for epoch in range(epochs):
            lr_t = self.learning_rate * np.exp(-epoch / epochs)
            for sample in data:
                bmu = self._find_bmu(sample)
                self.weights[bmu] += lr_t * (sample - self.weights[bmu])
    
    def get_winner_dist(self, sample):
        dists = np.linalg.norm(self.weights - sample, axis=2)
        bmu_idx = np.unravel_index(np.argmin(dists), dists.shape)
        return bmu_idx, np.min(dists)

def select_features_som(df, feat_cols, keep=[], n_select_per_cluster=2):
    """Use SOM to cluster and select representative features."""
    print("\n" + "=" * 60)
    print("STEP 4: SOM Feature Selection (Dimensionality Reduction)")
    print("=" * 60)
    
    data = df[feat_cols].T.values
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    
    som = SimpleSOM(4, 4, data_scaled.shape[1], learning_rate=0.5, sigma=1.5)
    som.fit(data_scaled, epochs=100)
    
    clusters = {}
    for i, f in enumerate(feat_cols):
        node, dist = som.get_winner_dist(data_scaled[i])
        if node not in clusters:
            clusters[node] = []
        clusters[node].append((f, dist))
    
    selected = []
    for node, items in clusters.items():
        items.sort(key=lambda x: x[1])
        selected.extend([item[0] for item in items[:n_select_per_cluster]])
    
    final = list(set(selected + keep))
    
    print(f"  Initial features: {len(feat_cols)}")
    print(f"  Number of clusters: {len(clusters)}")
    print(f"  Selected features: {len(final)}")
    
    return final

### 5.2 Multicollinearity Analysis (VIF)

In [ ]:
# ==========================================
# 6. MULTI-COLLINEARITY ANALYSIS
# ==========================================
def check_multicollinearity(df, features, threshold=10.0):
    """Calculate VIF to detect multicollinearity."""
    print("\n" + "=" * 60)
    print("STEP 5: Multi-collinearity Analysis (VIF)")
    print("=" * 60)
    
    try:
        from statsmodels.stats.outliers_influence import variance_inflation_factor
        
        X = df[features].copy()
        X = X.replace([np.inf, -np.inf], np.nan).dropna()
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=features)
        
        vif_data = pd.DataFrame()
        vif_data["Feature"] = features
        vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]
        vif_data = vif_data.sort_values('VIF', ascending=False)
        
        high_vif = vif_data[vif_data['VIF'] > threshold]
        print(f"  VIF Threshold: {threshold}")
        print(f"  Features with high VIF (>{threshold}): {len(high_vif)}")
        
        features_to_keep = vif_data[vif_data['VIF'] <= threshold * 2]['Feature'].tolist()
        return features_to_keep, vif_data
    except ImportError:
        print("  Warning: statsmodels not installed. Skipping VIF analysis.")
        return features, None

### 5.3 Outlier Detection

In [ ]:
# ==========================================
# 7. OUTLIER DETECTION
# ==========================================
def detect_and_handle_outliers(df, features, threshold=3.0):
    """Detect and handle outliers using IQR with Winsorization."""
    print("\n" + "=" * 60)
    print("STEP 6: Outlier Detection")
    print("=" * 60)
    
    df = df.copy()
    total_outliers = 0
    
    for feat in features:
        if feat not in df.columns:
            continue
        Q1 = df[feat].quantile(0.25)
        Q3 = df[feat].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR
        
        outliers = ((df[feat] < lower) | (df[feat] > upper)).sum()
        total_outliers += outliers
        
        df.loc[df[feat] < lower, feat] = lower
        df.loc[df[feat] > upper, feat] = upper
    
    print(f"  Method: IQR with threshold={threshold}")
    print(f"  Total outliers winsorized: {total_outliers}")
    
    return df

## 6. Exploratory Data Analysis (EDA)

In [ ]:
# ==========================================
# 8. EDA
# ==========================================
def run_comprehensive_eda(df, features, target):
    """Comprehensive EDA."""
    print("\n" + "=" * 60)
    print("STEP 7: Exploratory Data Analysis (EDA)")
    print("=" * 60)
    
    target_counts = df[target].value_counts(normalize=True) * 100
    print(f"\n  Target Distribution:")
    print(f"    Class 0: {target_counts.get(0, 0):.2f}%")
    print(f"    Class 1: {target_counts.get(1, 0):.2f}%")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Target distribution
    ax1 = axes[0, 0]
    colors = ['#ff6b6b', '#4ecdc4']
    bars = ax1.bar(['Down/Neutral', 'Up'], [target_counts.get(0, 0), target_counts.get(1, 0)], color=colors)
    ax1.set_title('Target Distribution', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Percentage (%)')
    for bar, val in zip(bars, [target_counts.get(0, 0), target_counts.get(1, 0)]):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', ha='center')
    
    # Correlation heatmap
    ax2 = axes[0, 1]
    valid_features = [f for f in features if f in df.columns][:15]
    corr_matrix = df[valid_features].corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='RdBu_r', center=0, ax=ax2)
    ax2.set_title('Feature Correlation Heatmap', fontsize=12, fontweight='bold')
    
    # Feature distributions
    ax3 = axes[1, 0]
    plot_features = [f for f in ['returns_1d', 'RSI', 'volatility_10'] if f in df.columns]
    if plot_features:
        df_melt = df[plot_features + [target]].melt(id_vars=target, var_name='Feature', value_name='Value')
        sns.boxplot(data=df_melt, x='Feature', y='Value', hue=target, ax=ax3, palette='Set2')
        ax3.set_title('Feature Distribution by Target', fontsize=12, fontweight='bold')
    
    # Returns distribution
    ax4 = axes[1, 1]
    if 'returns_1d' in df.columns:
        df[df[target] == 0]['returns_1d'].hist(ax=ax4, bins=50, alpha=0.5, label='Class 0', color='#ff6b6b')
        df[df[target] == 1]['returns_1d'].hist(ax=ax4, bins=50, alpha=0.5, label='Class 1', color='#4ecdc4')
        ax4.set_title('1-Day Returns by Class', fontsize=12, fontweight='bold')
        ax4.legend()
    
    plt.tight_layout()
    plt.savefig('eda_overview.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Pairwise scatter
    print("\n  Generating pairwise scatter plots...")
    scatter_features = [f for f in ['returns_1d', 'RSI', 'volatility_10', 'MACD'] if f in df.columns][:4]
    if len(scatter_features) >= 3:
        scatter_df = df[scatter_features + [target]].copy()
        scatter_df[target] = scatter_df[target].astype(str)
        g = sns.pairplot(scatter_df, hue=target, diag_kind='kde', 
                        palette={'0': '#ff6b6b', '1': '#4ecdc4'},
                        plot_kws={'alpha': 0.5, 's': 20})
        plt.savefig('pairwise_scatter.png', dpi=150, bbox_inches='tight')
        plt.show()

## 7. Model Definition

### 7.1 Hyperparameter Optimization

In [ ]:
# ==========================================
# 9. HYPERPARAMETER OPTIMIZATION
# ==========================================
def optimize_base_learners(X_train, y_train, class_weights, n_iter=20, cv=5):
    """Optimize hyperparameters for all base learners."""
    print("\n" + "=" * 60)
    print("STEP 8: Hyperparameter Optimization (All Base Learners)")
    print("=" * 60)
    
    results = {}
    tscv = TimeSeriesSplit(n_splits=cv)
    
    # 1. Logistic Regression
    print("\n  [1/5] Tuning Logistic Regression...")
    lr_param = {'C': [0.01, 0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
    lr_search = RandomizedSearchCV(
        LogisticRegression(class_weight=class_weights, random_state=42, max_iter=1000),
        lr_param, n_iter=8, scoring='roc_auc', cv=tscv, random_state=42, n_jobs=-1
    )
    lr_search.fit(X_train, y_train)
    results['lr'] = lr_search.best_params_
    print(f"    Best: {lr_search.best_params_}, AUC: {lr_search.best_score_:.4f}")
    
    # 2. Random Forest
    print("\n  [2/5] Tuning Random Forest...")
    rf_param = {
        'n_estimators': sp_randint(100, 300),
        'max_depth': sp_randint(5, 15),
        'min_samples_split': sp_randint(5, 20),
        'min_samples_leaf': sp_randint(2, 10)
    }
    rf_search = RandomizedSearchCV(
        RandomForestClassifier(class_weight=class_weights, random_state=42, n_jobs=-1),
        rf_param, n_iter=n_iter, scoring='roc_auc', cv=tscv, random_state=42, n_jobs=-1
    )
    rf_search.fit(X_train, y_train)
    results['rf'] = rf_search.best_params_
    print(f"    Best: {rf_search.best_params_}, AUC: {rf_search.best_score_:.4f}")
    
    # 3. LightGBM
    print("\n  [3/5] Tuning LightGBM...")
    lgbm_param = {
        'n_estimators': sp_randint(100, 300),
        'learning_rate': sp_uniform(0.01, 0.15),
        'max_depth': sp_randint(3, 10),
        'num_leaves': sp_randint(20, 50),
        'min_child_samples': sp_randint(20, 50)
    }
    lgbm_search = RandomizedSearchCV(
        LGBMClassifier(class_weight=class_weights, random_state=42, verbose=-1, n_jobs=-1),
        lgbm_param, n_iter=n_iter, scoring='roc_auc', cv=tscv, random_state=42, n_jobs=-1
    )
    lgbm_search.fit(X_train, y_train)
    results['lgbm'] = lgbm_search.best_params_
    print(f"    Best: {lgbm_search.best_params_}, AUC: {lgbm_search.best_score_:.4f}")
    
    # 4. XGBoost
    print("\n  [4/5] Tuning XGBoost...")
    ratio = class_weights.get(1, 1) / class_weights.get(0, 1) if class_weights else 1
    xgb_param = {
        'n_estimators': sp_randint(100, 300),
        'learning_rate': sp_uniform(0.01, 0.15),
        'max_depth': sp_randint(3, 10),
        'min_child_weight': sp_randint(1, 10)
    }
    xgb_search = RandomizedSearchCV(
        XGBClassifier(scale_pos_weight=ratio, random_state=42, n_jobs=-1, eval_metric='logloss', verbosity=0),
        xgb_param, n_iter=n_iter, scoring='roc_auc', cv=tscv, random_state=42, n_jobs=-1
    )
    xgb_search.fit(X_train, y_train)
    results['xgb'] = xgb_search.best_params_
    print(f"    Best: {xgb_search.best_params_}, AUC: {xgb_search.best_score_:.4f}")
    
    # 5. Gradient Boosting (instead of SVC for better probability calibration)
    print("\n  [5/5] Tuning Gradient Boosting...")
    gb_param = {
        'n_estimators': sp_randint(100, 300),
        'learning_rate': sp_uniform(0.01, 0.15),
        'max_depth': sp_randint(3, 8),
        'min_samples_split': sp_randint(5, 20)
    }
    gb_search = RandomizedSearchCV(
        GradientBoostingClassifier(random_state=42),
        gb_param, n_iter=n_iter, scoring='roc_auc', cv=tscv, random_state=42, n_jobs=-1
    )
    gb_search.fit(X_train, y_train)
    results['gb'] = gb_search.best_params_
    print(f"    Best: {gb_search.best_params_}, AUC: {gb_search.best_score_:.4f}")
    
    print("\n  Hyperparameter optimization complete.")
    return results

### 7.2 Blending Ensemble Class

In [ ]:
# ==========================================
# 10. BLENDING ENSEMBLE CLASS (CORRECT IMPLEMENTATION)
# ==========================================
class BlendingEnsemble:
    """
    CORRECT Blending Ensemble Implementation.
    
    Architecture:
    1. Split training data: base_train (70%) + blend_holdout (30%)
    2. Train base learners on base_train
    3. Generate meta-features: base learners predict on blend_holdout
    4. Train meta-learner on meta-features
    5. Prediction: base learners predict on test → meta-learner combines
    
    This is DIFFERENT from Stacking which uses K-fold CV for meta-features.
    """
    
    def __init__(self, base_learners, meta_learner, blend_ratio=0.7):
        """
        Parameters:
        -----------
        base_learners : list of (name, estimator) tuples
        meta_learner : estimator for combining base predictions
        blend_ratio : float, proportion of training data for base learners
        """
        self.base_learners = base_learners
        self.meta_learner = meta_learner
        self.blend_ratio = blend_ratio
        self.trained_base_learners = []
        self.trained_meta_learner = None
        
    def fit(self, X, y):
        """
        Fit the blending ensemble.
        
        Step 1: Split training data
        Step 2: Train base learners on base_train
        Step 3: Generate meta-features on blend_holdout
        Step 4: Train meta-learner
        """
        n_samples = len(X)
        split_idx = int(n_samples * self.blend_ratio)
        
        # Step 1: Split data
        X_base_train = X[:split_idx]
        y_base_train = y[:split_idx]
        X_blend_holdout = X[split_idx:]
        y_blend_holdout = y[split_idx:]
        
        print(f"    Blending split: {len(X_base_train)} base_train, {len(X_blend_holdout)} blend_holdout")
        
        # Step 2: Train base learners
        self.trained_base_learners = []
        for name, learner in self.base_learners:
            model = clone(learner)
            model.fit(X_base_train, y_base_train)
            self.trained_base_learners.append((name, model))
        
        # Step 3: Generate meta-features
        meta_features = self._generate_meta_features(X_blend_holdout)
        
        # Step 4: Train meta-learner
        self.trained_meta_learner = clone(self.meta_learner)
        self.trained_meta_learner.fit(meta_features, y_blend_holdout)
        
        return self
    
    def _generate_meta_features(self, X):
        """Generate meta-features from base learner predictions."""
        meta_features = []
        for name, model in self.trained_base_learners:
            if hasattr(model, 'predict_proba'):
                proba = model.predict_proba(X)[:, 1]
            else:
                proba = model.predict(X)
            meta_features.append(proba)
        return np.column_stack(meta_features)
    
    def predict_proba(self, X):
        """Predict probabilities using blending."""
        meta_features = self._generate_meta_features(X)
        if hasattr(self.trained_meta_learner, 'predict_proba'):
            return self.trained_meta_learner.predict_proba(meta_features)
        else:
            pred = self.trained_meta_learner.predict(meta_features)
            return np.column_stack([1 - pred, pred])
    
    def predict(self, X):
        """Predict classes."""
        proba = self.predict_proba(X)
        return (proba[:, 1] >= 0.5).astype(int)
    
    def get_base_predictions(self, X):
        """Get individual base learner predictions (for analysis)."""
        predictions = {}
        for name, model in self.trained_base_learners:
            if hasattr(model, 'predict_proba'):
                predictions[name] = model.predict_proba(X)[:, 1]
            else:
                predictions[name] = model.predict(X)
        return predictions

def create_blending_ensemble(optimized_params, class_weights, blend_ratio=0.7):
    """
    Create a BlendingEnsemble with 5 optimized base learners.
    """
    ratio = class_weights.get(1, 1) / class_weights.get(0, 1) if class_weights else 1
    
    # Base learners with optimized parameters
    base_learners = [
        ('LR', LogisticRegression(
            **{k: v for k, v in optimized_params.get('lr', {}).items()},
            class_weight=class_weights, random_state=42, max_iter=1000
        )),
        ('RF', RandomForestClassifier(
            **optimized_params.get('rf', {}),
            class_weight=class_weights, random_state=42, n_jobs=-1
        )),
        ('LGBM', LGBMClassifier(
            **optimized_params.get('lgbm', {}),
            class_weight=class_weights, random_state=42, verbose=-1, n_jobs=-1
        )),
        ('XGB', XGBClassifier(
            **optimized_params.get('xgb', {}),
            scale_pos_weight=ratio, random_state=42, n_jobs=-1, 
            eval_metric='logloss', verbosity=0
        )),
        ('GB', GradientBoostingClassifier(
            **optimized_params.get('gb', {}),
            random_state=42
        ))
    ]
    
    # Meta-learner
    meta_learner = LogisticRegression(solver='lbfgs', random_state=42, max_iter=1000)
    
    return BlendingEnsemble(base_learners, meta_learner, blend_ratio=blend_ratio)

### 7.3 Rolling Prediction

In [ ]:
# ==========================================
# 11. ROLLING BLENDING PREDICTION
# ==========================================
def train_predict_rolling_blending(X_all, y_all, df_index, optimized_params,
                                    initial_train_size=500, step_size=22,
                                    blend_ratio=0.7, min_train_samples=200):
    """
    Rolling window prediction using BLENDING ensemble.
    
    For each rolling window:
    1. Use training window data
    2. Apply blending: split into base_train (70%) and blend_holdout (30%)
    3. Train base learners on base_train
    4. Generate meta-features on blend_holdout
    5. Train meta-learner
    6. Predict on test window
    """
    print("\n" + "=" * 60)
    print("STEP 9: Rolling Blending Prediction")
    print("=" * 60)
    print(f"  Initial train size: {initial_train_size}")
    print(f"  Step size: {step_size}")
    print(f"  Blend ratio: {blend_ratio:.0%} base_train / {1-blend_ratio:.0%} blend_holdout")
    
    n_samples = len(y_all)
    predictions = []
    indices = []
    base_predictions = []
    actuals = []
    skipped_windows = 0
    
    # Progress bar
    total_windows = (n_samples - initial_train_size) // step_size + 1
    
    for start_idx in tqdm(range(initial_train_size, n_samples, step_size), 
                          desc="Rolling Blending", total=total_windows):
        end_idx = min(start_idx + step_size, n_samples)
        
        # Training window (expanding window)
        train_start = max(0, start_idx - initial_train_size)
        train_end = start_idx
        
        X_train_window = X_all[train_start:train_end]
        y_train_window = y_all[train_start:train_end]
        X_test_window = X_all[start_idx:end_idx]
        y_test_window = y_all[start_idx:end_idx]
        
        # Skip if insufficient data
        if len(X_train_window) < min_train_samples or len(X_test_window) == 0:
            skipped_windows += 1
            continue
        
        # Check class distribution
        unique_classes = np.unique(y_train_window)
        if len(unique_classes) < 2:
            skipped_windows += 1
            continue
        
        # Minimum samples per class
        class_counts = np.bincount(y_train_window.astype(int))
        if min(class_counts) < 30:  # Need enough for blending split
            skipped_windows += 1
            continue
        
        try:
            # Calculate class weights
            cw = dict(zip(unique_classes,
                         class_weight.compute_class_weight('balanced',
                                                          classes=unique_classes,
                                                          y=y_train_window)))
            
            # Create and train blending ensemble
            blending_clf = create_blending_ensemble(optimized_params, cw, blend_ratio=blend_ratio)
            blending_clf.fit(X_train_window, y_train_window)
            
            # Predict
            pred_proba = blending_clf.predict_proba(X_test_window)[:, 1]
            predictions.extend(pred_proba)
            indices.extend(range(start_idx, end_idx))
            actuals.extend(y_test_window)
            
            # Baseline (simple LR trained on full training window)
            lr_baseline = LogisticRegression(solver='liblinear', class_weight=cw,
                                            random_state=42, max_iter=1000)
            lr_baseline.fit(X_train_window, y_train_window)
            baseline_pred = lr_baseline.predict_proba(X_test_window)[:, 1]
            base_predictions.extend(baseline_pred)
            
        except Exception as e:
            print(f"\n  Warning: Window failed: {str(e)[:50]}")
            skipped_windows += 1
            continue
    
    print(f"\n  Total predictions: {len(predictions)}")
    print(f"  Skipped windows: {skipped_windows}")
    
    return (np.array(predictions), np.array(indices),
            np.array(base_predictions), np.array(actuals))

## 8. Model Evaluation

In [ ]:
# ==========================================
# 12. EVALUATION FUNCTIONS
# ==========================================
def evaluate_model(y_true, y_pred_prob, y_pred_binary, model_name="Model"):
    """Comprehensive model evaluation."""
    results = {
        'AUC': roc_auc_score(y_true, y_pred_prob),
        'Balanced_Accuracy': balanced_accuracy_score(y_true, y_pred_binary),
        'MCC': matthews_corrcoef(y_true, y_pred_binary),
        'F1': f1_score(y_true, y_pred_binary),
        'Brier': brier_score_loss(y_true, y_pred_prob),
        'AP': average_precision_score(y_true, y_pred_prob)
    }
    
    print(f"\n  {model_name} Evaluation:")
    print(f"    AUC-ROC: {results['AUC']:.4f}")
    print(f"    Average Precision: {results['AP']:.4f}")
    print(f"    Balanced Accuracy: {results['Balanced_Accuracy']:.4f}")
    print(f"    MCC: {results['MCC']:.4f}")
    print(f"    F1 Score: {results['F1']:.4f}")
    print(f"    Brier Score: {results['Brier']:.4f}")
    
    return results

def plot_confusion_matrix_comparison(y_true, y_pred_ensemble, y_pred_baseline):
    """Plot confusion matrices."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    cm_ens = confusion_matrix(y_true, y_pred_ensemble)
    ConfusionMatrixDisplay(cm_ens, display_labels=['Down', 'Up']).plot(ax=axes[0], cmap='Blues')
    axes[0].set_title('Blending Ensemble', fontsize=12, fontweight='bold')
    
    cm_base = confusion_matrix(y_true, y_pred_baseline)
    ConfusionMatrixDisplay(cm_base, display_labels=['Down', 'Up']).plot(ax=axes[1], cmap='Oranges')
    axes[1].set_title('Baseline (Logistic Regression)', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Backtesting

In [ ]:
# ==========================================
# 13. BACKTESTING (CORRECTED)
# ==========================================
def calculate_backtest_metrics(returns, name="Strategy"):
    """Calculate backtest metrics with CORRECT max drawdown."""
    active_returns = returns[returns != 0]
    
    if len(active_returns) < 10:
        return {'Sharpe': 0, 'Sortino': 0, 'Total_Return': 0,
                'Max_Drawdown': 0, 'Win_Rate': 0, 'Num_Trades': 0, 'Profit_Factor': 0}
    
    ann_factor = np.sqrt(252 / 5)
    
    # Sharpe
    sharpe = np.mean(active_returns) / (np.std(active_returns) + 1e-9) * ann_factor
    
    # Sortino
    downside = active_returns[active_returns < 0]
    sortino = np.mean(active_returns) / (np.std(downside) + 1e-9) * ann_factor if len(downside) > 0 else sharpe
    
    # Total Return
    total_return = (1 + returns).prod() - 1
    
    # CORRECTED Max Drawdown
    cumulative = (1 + returns).cumprod()
    peak = np.maximum.accumulate(cumulative)
    drawdown = (peak - cumulative) / (peak + 1e-9)
    max_drawdown = np.max(drawdown)
    
    # Win Rate
    win_rate = np.mean(active_returns > 0)
    
    # Profit Factor
    gross_profit = np.sum(active_returns[active_returns > 0])
    gross_loss = np.abs(np.sum(active_returns[active_returns < 0]))
    profit_factor = gross_profit / (gross_loss + 1e-9)
    
    return {
        'Sharpe': sharpe, 'Sortino': sortino, 'Total_Return': total_return,
        'Max_Drawdown': max_drawdown, 'Win_Rate': win_rate,
        'Num_Trades': len(active_returns), 'Profit_Factor': profit_factor
    }

def run_backtest(test_dates, test_returns, y_pred_prob, y_baseline_prob,
                 threshold=0.5, apply_trend_filter=True, df=None, aligned_indices=None):
    """Run backtest."""
    print("\n" + "=" * 60)
    print("STEP 10: Backtesting")
    print("=" * 60)
    
    signal_ensemble = (y_pred_prob >= threshold).astype(int)
    signal_baseline = (y_baseline_prob >= threshold).astype(int)
    
    ret_ensemble = test_returns * signal_ensemble
    ret_baseline = test_returns * signal_baseline
    ret_market = test_returns.copy()
    
    if apply_trend_filter and df is not None and aligned_indices is not None:
        ma200 = df['Close'].rolling(200).mean().iloc[aligned_indices].values
        price = df['Close'].iloc[aligned_indices].values
        min_len = min(len(ma200), len(ret_ensemble))
        trend_filter = (price[:min_len] > ma200[:min_len]).astype(int)
        ret_ensemble[:min_len] *= trend_filter
        ret_baseline[:min_len] *= trend_filter
        print(f"  Applied 200-day MA trend filter (uptrend: {np.mean(trend_filter)*100:.1f}%)")
    
    metrics_ens = calculate_backtest_metrics(ret_ensemble, "Ensemble")
    metrics_base = calculate_backtest_metrics(ret_baseline, "Baseline")
    metrics_mkt = calculate_backtest_metrics(ret_market, "Market")
    
    print(f"\n  Performance Summary:")
    print(f"  {'Metric':<18} {'Ensemble':>12} {'Baseline':>12} {'Market':>12}")
    print(f"  {'-'*54}")
    print(f"  {'Sharpe Ratio':<18} {metrics_ens['Sharpe']:>12.3f} {metrics_base['Sharpe']:>12.3f} {metrics_mkt['Sharpe']:>12.3f}")
    print(f"  {'Sortino Ratio':<18} {metrics_ens['Sortino']:>12.3f} {metrics_base['Sortino']:>12.3f} {metrics_mkt['Sortino']:>12.3f}")
    print(f"  {'Total Return':<18} {metrics_ens['Total_Return']:>11.2%} {metrics_base['Total_Return']:>11.2%} {metrics_mkt['Total_Return']:>11.2%}")
    print(f"  {'Max Drawdown':<18} {metrics_ens['Max_Drawdown']:>11.2%} {metrics_base['Max_Drawdown']:>11.2%} {metrics_mkt['Max_Drawdown']:>11.2%}")
    print(f"  {'Win Rate':<18} {metrics_ens['Win_Rate']:>11.2%} {metrics_base['Win_Rate']:>11.2%} {metrics_mkt['Win_Rate']:>11.2%}")
    print(f"  {'Profit Factor':<18} {metrics_ens['Profit_Factor']:>12.2f} {metrics_base['Profit_Factor']:>12.2f} {metrics_mkt['Profit_Factor']:>12.2f}")
    
    return ret_ensemble, ret_baseline, ret_market, metrics_ens, metrics_base


## 10. Visualization

In [ ]:
# ==========================================
# 14. VISUALIZATION FUNCTIONS
# ==========================================
def plot_cumulative_returns(test_dates, ret_ensemble, ret_baseline, ret_market):
    """Plot cumulative returns."""
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Additive cumulative
    ax1 = axes[0]
    ax1.plot(test_dates, np.cumsum(ret_market), label='Market', color='gray', alpha=0.6)
    ax1.plot(test_dates, np.cumsum(ret_baseline), label='Baseline (LR)', color='orange', linestyle='--')
    ax1.plot(test_dates, np.cumsum(ret_ensemble), label='Blending Ensemble', color='blue', linewidth=2)
    ax1.set_title('Cumulative Returns (Additive)', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    # Multiplicative (wealth)
    ax2 = axes[1]
    ax2.plot(test_dates, (1 + ret_market).cumprod(), label='Market', color='gray', alpha=0.6)
    ax2.plot(test_dates, (1 + ret_baseline).cumprod(), label='Baseline', color='orange', linestyle='--')
    ax2.plot(test_dates, (1 + ret_ensemble).cumprod(), label='Blending Ensemble', color='blue', linewidth=2)
    ax2.set_title('Portfolio Value ($1 Initial)', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('cumulative_returns.png', dpi=150, bbox_inches='tight')
    plt.show()

def plot_drawdown(test_dates, ret_ensemble, ret_baseline, ret_market):
    """Plot drawdown comparison."""
    def calc_dd(returns):
        cum = (1 + returns).cumprod()
        peak = np.maximum.accumulate(cum)
        return (peak - cum) / peak
    
    plt.figure(figsize=(14, 6))
    plt.fill_between(test_dates, 0, -calc_dd(ret_market), alpha=0.3, color='gray', label='Market')
    plt.fill_between(test_dates, 0, -calc_dd(ret_baseline), alpha=0.3, color='orange', label='Baseline')
    plt.fill_between(test_dates, 0, -calc_dd(ret_ensemble), alpha=0.5, color='blue', label='Ensemble')
    plt.title('Drawdown Comparison', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    plt.tight_layout()
    plt.savefig('drawdown_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

def plot_rolling_metrics(test_dates, ret_ensemble, ret_baseline, ret_market, window=63):
    """Plot rolling metrics."""
    ann_factor = np.sqrt(252 / 5)
    ens = pd.Series(ret_ensemble, index=test_dates)
    base = pd.Series(ret_baseline, index=test_dates)
    mkt = pd.Series(ret_market, index=test_dates)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Rolling Sharpe
    ax1 = axes[0]
    ax1.plot(mkt.rolling(window).mean() / (mkt.rolling(window).std() + 1e-9) * ann_factor,
             label='Market', color='gray', alpha=0.5)
    ax1.plot(base.rolling(window).mean() / (base.rolling(window).std() + 1e-9) * ann_factor,
             label='Baseline', color='orange', linestyle='--')
    ax1.plot(ens.rolling(window).mean() / (ens.rolling(window).std() + 1e-9) * ann_factor,
             label='Ensemble', color='blue')
    ax1.axhline(0, color='red', linestyle=':', alpha=0.5)
    ax1.set_title(f'Rolling Sharpe Ratio ({window}-day)', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Rolling Win Rate
    ax2 = axes[1]
    ax2.plot((mkt > 0).rolling(window).mean(), label='Market', color='gray', alpha=0.5)
    ax2.plot((base > 0).rolling(window).mean(), label='Baseline', color='orange', linestyle='--')
    ax2.plot((ens > 0).rolling(window).mean(), label='Ensemble', color='blue')
    ax2.axhline(0.5, color='red', linestyle=':', alpha=0.5)
    ax2.set_title(f'Rolling Win Rate ({window}-day)', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    plt.tight_layout()
    plt.savefig('rolling_metrics.png', dpi=150, bbox_inches='tight')
    plt.show()

def plot_roc_pr_curves(y_true, y_pred_ensemble, y_pred_baseline):
    """Plot ROC and PR curves."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # ROC
    ax1 = axes[0]
    fpr_e, tpr_e, _ = roc_curve(y_true, y_pred_ensemble)
    fpr_b, tpr_b, _ = roc_curve(y_true, y_pred_baseline)
    auc_e = roc_auc_score(y_true, y_pred_ensemble)
    auc_b = roc_auc_score(y_true, y_pred_baseline)
    ax1.plot(fpr_e, tpr_e, label=f'Ensemble (AUC={auc_e:.4f})', color='blue', linewidth=2)
    ax1.plot(fpr_b, tpr_b, label=f'Baseline (AUC={auc_b:.4f})', color='orange', linestyle='--')
    ax1.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax1.set_xlabel('FPR')
    ax1.set_ylabel('TPR')
    ax1.set_title('ROC Curve', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # PR
    ax2 = axes[1]
    prec_e, rec_e, _ = precision_recall_curve(y_true, y_pred_ensemble)
    prec_b, rec_b, _ = precision_recall_curve(y_true, y_pred_baseline)
    ap_e = average_precision_score(y_true, y_pred_ensemble)
    ap_b = average_precision_score(y_true, y_pred_baseline)
    ax2.plot(rec_e, prec_e, label=f'Ensemble (AP={ap_e:.4f})', color='blue', linewidth=2)
    ax2.plot(rec_b, prec_b, label=f'Baseline (AP={ap_b:.4f})', color='orange', linestyle='--')
    ax2.axhline(np.mean(y_true), color='gray', linestyle=':', label=f'Baseline ({np.mean(y_true):.2%})')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curve', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('roc_pr_curves.png', dpi=150, bbox_inches='tight')
    plt.show()

def plot_feature_importance(X_train, y_train, feature_names, top_n=15):
    """Plot feature importance."""
    print("\n  Generating feature importance plot...")
    rf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=True).tail(top_n)
    
    plt.figure(figsize=(10, 8))
    plt.barh(importance['Feature'], importance['Importance'], color='steelblue')
    plt.title('Feature Importance (Random Forest)', fontsize=12, fontweight='bold')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
    plt.show()

def generate_summary_table(eval_ens, eval_base, metrics_ens, metrics_base):
    """Generate summary table."""
    summary = pd.DataFrame({
        'Metric': ['AUC-ROC', 'Average Precision', 'Balanced Accuracy', 'MCC', 'F1 Score', 'Brier Score',
                   'Sharpe Ratio', 'Sortino Ratio', 'Total Return', 'Max Drawdown', 'Win Rate', 'Profit Factor'],
        'Blending Ensemble': [
            f"{eval_ens['AUC']:.4f}", f"{eval_ens['AP']:.4f}", f"{eval_ens['Balanced_Accuracy']:.4f}",
            f"{eval_ens['MCC']:.4f}", f"{eval_ens['F1']:.4f}", f"{eval_ens['Brier']:.4f}",
            f"{metrics_ens['Sharpe']:.3f}", f"{metrics_ens['Sortino']:.3f}", f"{metrics_ens['Total_Return']:.2%}",
            f"{metrics_ens['Max_Drawdown']:.2%}", f"{metrics_ens['Win_Rate']:.2%}", f"{metrics_ens['Profit_Factor']:.2f}"
        ],
        'Baseline (LR)': [
            f"{eval_base['AUC']:.4f}", f"{eval_base['AP']:.4f}", f"{eval_base['Balanced_Accuracy']:.4f}",
            f"{eval_base['MCC']:.4f}", f"{eval_base['F1']:.4f}", f"{eval_base['Brier']:.4f}",
            f"{metrics_base['Sharpe']:.3f}", f"{metrics_base['Sortino']:.3f}", f"{metrics_base['Total_Return']:.2%}",
            f"{metrics_base['Max_Drawdown']:.2%}", f"{metrics_base['Win_Rate']:.2%}", f"{metrics_base['Profit_Factor']:.2f}"
        ]
    })
    
    print("\n" + "=" * 60)
    print("SUMMARY TABLE")
    print("=" * 60)
    print(summary.to_string(index=False))
    return summary

def create_methods_table():
    """Create numerical methods table for report."""
    methods = pd.DataFrame({
        'Method': ['Self-Organizing Map (SOM)', 'Fractional Differentiation', 'VIF Analysis',
                   'Logistic Regression', 'Random Forest', 'LightGBM', 'XGBoost', 'Gradient Boosting',
                   'Blending Ensemble', 'RandomizedSearchCV', 'TimeSeriesSplit'],
        'Purpose': ['Feature clustering & selection', 'Stationarity with memory', 'Multicollinearity detection',
                    'Base learner 1', 'Base learner 2', 'Base learner 3', 'Base learner 4', 'Base learner 5',
                    'Meta-learner combination', 'Hyperparameter tuning', 'Temporal cross-validation'],
        'Implementation': ['Custom class', 'Custom function', 'statsmodels', 'sklearn', 'sklearn',
                          'lightgbm', 'xgboost', 'sklearn', 'Custom BlendingEnsemble class', 'sklearn', 'sklearn']
    })
    
    print("\n" + "=" * 60)
    print("NUMERICAL METHODS TABLE")
    print("=" * 60)
    print(methods.to_string(index=False))
    return methods

## 11. Main Execution

### 11.1 Main Function Definition

In [ ]:
# ==========================================
# 15. MAIN EXECUTION
# ==========================================
def run_final_project():
    """Main execution function."""
    print("\n" + "=" * 70)
    print("CQF FINAL PROJECT: BLENDING ENSEMBLE FOR CLASSIFICATION")
    print("=" * 70)
    print("Asset: SPY ETF")
    print("Prediction: 5-day forward return direction")
    print("Method: BLENDING Ensemble with 5 base learners")
    print("        (NOT Stacking - uses holdout split, not K-fold CV)")
    print("=" * 70)
    
    # File paths
    spy_train_path = 'SPYtrain.csv'
    spy_test_path = 'SPYtest.csv'
    gdp_path = 'GDPC1.csv'
    dgs10_path = 'DGS10.csv'
    
    # STEP 1: Load Data
    df, oos_start_date = load_and_merge(spy_train_path, spy_test_path, gdp_path, dgs10_path)
    
    # STEP 2: Feature Engineering
    df = create_features_enhanced(df)
    
    # STEP 3: Label Definition
    df = make_labels_momentum(df, lookahead=5, vol_factor=0.5)
    
    # STEP 4: Feature Selection
    drop_cols = ['Target', 'Close', 'Open', 'High', 'Low', 'Volume', 'GDPC1', 'DGS10',
                 'trade_ret', 'future_ret', 'prev_close', 'prev_open', 'prev_high', 'prev_low', 'prev_volume']
    feats_all = [c for c in df.columns if c not in drop_cols]
    
    core_feats = ['returns_1d', 'returns_5d', 'momentum_5d', 'volatility_10',
                  'money_flow', 'RSI', 'MACD', 'trend_strength', 'close_frac', 'bb_position']
    core_feats = [f for f in core_feats if f in feats_all]
    
    sel_feats = select_features_som(df, feats_all, keep=core_feats, n_select_per_cluster=2)
    
    # STEP 5: VIF Analysis
    sel_feats, vif_results = check_multicollinearity(df, sel_feats, threshold=10.0)
    
    # STEP 6: Outlier Detection
    df = detect_and_handle_outliers(df, sel_feats, threshold=3.0)
    
    # STEP 7: EDA
    run_comprehensive_eda(df.copy(), sel_feats, 'Target')
    
    # STEP 8: Data Preparation
    print("\n" + "=" * 60)
    print("Data Preparation")
    print("=" * 60)
    
    oos_idx = df.index.get_indexer([oos_start_date], method='nearest')[0]
    
    X_all = df[sel_feats].values
    y_all = df['Target'].values
    
    X_train = X_all[:oos_idx]
    y_train = y_all[:oos_idx]
    X_test = X_all[oos_idx:]
    y_test = y_all[oos_idx:]
    
    print(f"  Training samples: {len(X_train)}")
    print(f"  Test samples: {len(X_test)}")
    print(f"  Features: {len(sel_feats)}")
    print(f"  Train class dist: {np.bincount(y_train.astype(int))}")
    
    # Scale data (fit on train only!)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_all_scaled = np.vstack([X_train_scaled, X_test_scaled])
    y_all_combined = np.concatenate([y_train, y_test])
    
    # STEP 9: Hyperparameter Optimization
    classes = np.unique(y_train)
    class_weights = dict(zip(classes, class_weight.compute_class_weight('balanced', classes=classes, y=y_train)))
    print(f"\n  Class weights: {class_weights}")
    
    optimized_params = optimize_base_learners(X_train_scaled, y_train, class_weights, n_iter=20, cv=5)
    
    # STEP 10: Rolling Blending Prediction
    INITIAL_TRAIN_SIZE = len(X_train)
    STEP_SIZE = 22
    BLEND_RATIO = 0.7  # 70% base_train, 30% blend_holdout
    MIN_TRAIN = min(200, len(X_train) // 3)
    
    y_pred_prob, test_indices, baseline_prob, y_test_rolling = train_predict_rolling_blending(
        X_all_scaled, y_all_combined, df.index,
        optimized_params,
        initial_train_size=INITIAL_TRAIN_SIZE,
        step_size=STEP_SIZE,
        blend_ratio=BLEND_RATIO,
        min_train_samples=MIN_TRAIN
    )
    
    # STEP 11: Evaluation
    print("\n" + "=" * 60)
    print("Model Evaluation")
    print("=" * 60)
    
    # Find optimal threshold
    prec, rec, thresholds = precision_recall_curve(y_test_rolling, y_pred_prob)
    f1_scores = 2 * prec * rec / (prec + rec + 1e-9)
    best_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
    optimal_threshold = np.clip(optimal_threshold, 0.3, 0.7)
    print(f"\n  Optimal threshold: {optimal_threshold:.4f}")
    
    y_pred_binary = (y_pred_prob >= optimal_threshold).astype(int)
    baseline_binary = (baseline_prob >= optimal_threshold).astype(int)
    
    eval_ensemble = evaluate_model(y_test_rolling, y_pred_prob, y_pred_binary, "Blending Ensemble")
    eval_baseline = evaluate_model(y_test_rolling, baseline_prob, baseline_binary, "Baseline (LR)")
    
    print("\n  Classification Report (Ensemble):")
    print(classification_report(y_test_rolling, y_pred_binary, target_names=['Down', 'Up']))
    
    plot_confusion_matrix_comparison(y_test_rolling, y_pred_binary, baseline_binary)
    
    # STEP 12: Backtesting
    aligned_indices = test_indices
    test_returns = df['trade_ret'].iloc[aligned_indices].values
    test_dates = df.index[aligned_indices]
    
    min_len = min(len(test_returns), len(y_pred_binary))
    test_returns = test_returns[:min_len]
    test_dates = test_dates[:min_len]
    y_pred_prob_bt = y_pred_prob[:min_len]
    baseline_prob_bt = baseline_prob[:min_len]
    y_test_bt = y_test_rolling[:min_len]
    
    ret_ens, ret_base, ret_mkt, metrics_ens, metrics_base = run_backtest(
        test_dates, test_returns, y_pred_prob_bt, baseline_prob_bt,
        threshold=optimal_threshold, apply_trend_filter=True,
        df=df, aligned_indices=aligned_indices[:min_len]
    )
    
    # STEP 13: Visualizations
    print("\n" + "=" * 60)
    print("Generating Visualizations")
    print("=" * 60)
    
    plot_cumulative_returns(test_dates, ret_ens, ret_base, ret_mkt)
    plot_drawdown(test_dates, ret_ens, ret_base, ret_mkt)
    plot_rolling_metrics(test_dates, ret_ens, ret_base, ret_mkt, window=63)
    plot_roc_pr_curves(y_test_bt, y_pred_prob_bt, baseline_prob_bt)
    plot_feature_importance(X_train_scaled, y_train, sel_feats, top_n=15)
    
    # STEP 14: Summary
    summary = generate_summary_table(eval_ensemble, eval_baseline, metrics_ens, metrics_base)
    summary.to_csv('model_comparison_summary.csv', index=False)
    
    methods = create_methods_table()
    methods.to_csv('numerical_methods.csv', index=False)
    
    print("\n" + "=" * 70)
    print("PROJECT COMPLETE")
    print("=" * 70)
    print("\nKey Implementation Details:")
    print("  - BLENDING (not Stacking): Training data split into base_train + blend_holdout")
    print("  - 5 Base Learners: LR, RF, LightGBM, XGBoost, Gradient Boosting")
    print("  - Meta-Learner: Logistic Regression")
    print("  - All hyperparameters optimized via RandomizedSearchCV")

### 11.2 Run the Project

In [ ]:

if __name__ == '__main__':
    run_final_project()

---

## 12. Additional Analysis: Signal-to-Noise Ratio

In [ ]:
# ==========================================
# ADDITIONAL PLOTS - Standalone Version
# ==========================================
from scipy.stats import pearsonr, spearmanr

print("\n" + "=" * 70)
print("ADDITIONAL ANALYSIS - Loading Data")
print("=" * 70)

# Reload data
spy_train_path = 'SPYtrain.csv'
spy_test_path = 'SPYtest.csv'
gdp_path = 'GDPC1.csv'
dgs10_path = 'DGS10.csv'

df_train = pd.read_csv(spy_train_path, parse_dates=['Date'], index_col='Date').sort_index()
df_test = pd.read_csv(spy_test_path, parse_dates=['Date'], index_col='Date').sort_index()
oos_start_date = df_test.index.min()
df = pd.concat([df_train, df_test]).sort_index()

for col in ['Close/Last', 'Open', 'High', 'Low']:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = df[col].str.replace('$', '').astype(float)
if 'Close/Last' in df.columns:
    df.rename(columns={'Close/Last': 'Close'}, inplace=True)

gdp = pd.read_csv(gdp_path, parse_dates=['observation_date'])
gdp.columns = ['Date', 'GDPC1']
gdp.set_index('Date', inplace=True)
dgs10 = pd.read_csv(dgs10_path, parse_dates=['observation_date'])
dgs10.columns = ['Date', 'DGS10']
dgs10['DGS10'] = pd.to_numeric(dgs10['DGS10'], errors='coerce')
dgs10.set_index('Date', inplace=True)
df = df.join(gdp.reindex(df.index, method='ffill'))
df = df.join(dgs10.reindex(df.index, method='ffill')).ffill().bfill()

# Create features
df['prev_close'] = df['Close'].shift(1)
df['returns_1d'] = df['prev_close'].pct_change(1)
df['returns_5d'] = df['prev_close'].pct_change(5)
df['returns_10d'] = df['prev_close'].pct_change(10)
df['returns_20d'] = df['prev_close'].pct_change(20)
df['momentum_5d'] = df['prev_close'] - df['Close'].shift(6)
df['volatility_5'] = df['returns_1d'].rolling(5).std()
df['volatility_10'] = df['returns_1d'].rolling(10).std()
df['Vol_20'] = df['returns_1d'].rolling(20).std()

for window in [5, 10, 20, 50]:
    ma = df['prev_close'].rolling(window).mean()
    df[f'MA_{window}_ratio'] = df['prev_close'] / (ma + 1e-8)

exp1 = df['prev_close'].ewm(span=12, adjust=False).mean()
exp2 = df['prev_close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 - exp2

delta = df['prev_close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
df['RSI'] = 100 - (100 / (1 + gain / (loss + 1e-8)))

roll_mean = df['prev_close'].rolling(20).mean()
roll_std = df['prev_close'].rolling(20).std()
df['bb_position'] = (df['prev_close'] - roll_mean) / (2 * roll_std + 1e-8)

df['future_ret'] = df['Close'].pct_change(5).shift(-5)
df['Target'] = np.where(df['future_ret'] > df['Vol_20'] * 0.5, 1, 0)
df = df.dropna()

features = ['returns_1d', 'returns_5d', 'returns_10d', 'returns_20d',
            'momentum_5d', 'volatility_5', 'volatility_10', 'Vol_20',
            'MA_5_ratio', 'MA_10_ratio', 'MA_20_ratio', 'MA_50_ratio',
            'MACD', 'RSI', 'bb_position']
features = [f for f in features if f in df.columns]

oos_idx = df.index.get_indexer([oos_start_date], method='nearest')[0]
X_train = df[features].values[:oos_idx]
y_train = df['Target'].values[:oos_idx]
X_test = df[features].values[oos_idx:]
y_test = df['Target'].values[oos_idx:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classes = np.unique(y_train)
class_weights = dict(zip(classes, class_weight.compute_class_weight('balanced', classes=classes, y=y_train)))

print(f"  Train: {len(X_train)}, Test: {len(X_test)}, Features: {len(features)}")

# ==========================================
# PLOT 1: Base Learners Comparison
# ==========================================
print("\n" + "=" * 60)
print("Plot 1: Base Learners Individual Performance")
print("=" * 60)

ratio = class_weights.get(1, 1) / class_weights.get(0, 1)
learners = {
    'Logistic Regression': LogisticRegression(class_weight=class_weights, random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=10, class_weight=class_weights, random_state=42, n_jobs=-1),
    'LightGBM': LGBMClassifier(n_estimators=200, max_depth=6, class_weight=class_weights, random_state=42, verbose=-1, n_jobs=-1),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=6, scale_pos_weight=ratio, random_state=42, n_jobs=-1, eval_metric='logloss', verbosity=0),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, max_depth=5, random_state=42)
}

results = {}
predictions = {}
for name, model in learners.items():
    print(f"  Training {name}...")
    model.fit(X_train_scaled, y_train)
    y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)
    results[name] = auc
    predictions[name] = y_pred_prob
    print(f"    AUC: {auc:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
names = list(results.keys())
aucs = list(results.values())
colors = ['#3498db', '#2ecc71', '#9b59b6', '#e74c3c', '#f39c12']
bars = ax1.barh(names, aucs, color=colors, edgecolor='black', linewidth=0.5)
ax1.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Random (0.5)')
ax1.set_xlim(0.45, 0.60)
ax1.set_xlabel('AUC-ROC', fontsize=11)
ax1.set_title('Base Learners Individual AUC', fontsize=12, fontweight='bold')
for bar, auc_val in zip(bars, aucs):
    ax1.text(auc_val + 0.005, bar.get_y() + bar.get_height()/2, f'{auc_val:.4f}', va='center')
ax1.legend(loc='lower right')

ax2 = axes[1]
for (name, y_pred), color in zip(predictions.items(), colors):
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    ax2.plot(fpr, tpr, label=f'{name} ({results[name]:.4f})', color=color, linewidth=2)
ax2.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax2.set_xlabel('FPR'); ax2.set_ylabel('TPR')
ax2.set_title('ROC Curves - All Base Learners', fontsize=12, fontweight='bold')
ax2.legend(loc='lower right', fontsize=9); ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('base_learners_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Correlation heatmap
pred_df = pd.DataFrame(predictions)
corr_matrix = pred_df.corr()
print("\n  Prediction Correlation Matrix:")
print(corr_matrix.round(3))

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0.5, vmin=0, vmax=1, fmt='.3f')
plt.title('Base Learners Prediction Correlation\n(High = Low diversity)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('base_learners_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

# ==========================================
# PLOT 2: Signal-to-Noise Analysis
# ==========================================
print("\n" + "=" * 60)
print("Plot 2: Signal-to-Noise Ratio Analysis")
print("=" * 60)

correlations = []
for feat in features:
    pearson_r, _ = pearsonr(df[feat].fillna(0), df['Target'])
    correlations.append({'Feature': feat, 'Pearson_r': pearson_r, 'Abs_Pearson': abs(pearson_r)})
corr_df = pd.DataFrame(correlations).sort_values('Abs_Pearson', ascending=False)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

ax1 = axes[0, 0]
top_corr = corr_df.head(15)
colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in top_corr['Pearson_r']]
ax1.barh(top_corr['Feature'], top_corr['Pearson_r'], color=colors, edgecolor='black')
ax1.axvline(x=0, color='black', linewidth=0.5)
ax1.axvline(x=0.1, color='blue', linestyle='--', alpha=0.5, label='|r|=0.1')
ax1.axvline(x=-0.1, color='blue', linestyle='--', alpha=0.5)
ax1.set_xlabel('Pearson Correlation'); ax1.set_title('Feature-Target Correlations', fontweight='bold')
ax1.legend()

ax2 = axes[0, 1]
ax2.hist(corr_df['Pearson_r'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='red', linewidth=2)
ax2.axvline(x=corr_df['Pearson_r'].mean(), color='orange', linestyle='--', label=f'Mean: {corr_df["Pearson_r"].mean():.4f}')
ax2.set_xlabel('Pearson Correlation'); ax2.set_title('Correlation Distribution', fontweight='bold')
ax2.legend()

ax3 = axes[1, 0]
very_weak = (corr_df['Abs_Pearson'] < 0.05).sum()
weak = ((corr_df['Abs_Pearson'] >= 0.05) & (corr_df['Abs_Pearson'] < 0.1)).sum()
moderate = ((corr_df['Abs_Pearson'] >= 0.1) & (corr_df['Abs_Pearson'] < 0.2)).sum()
strong = (corr_df['Abs_Pearson'] >= 0.2).sum()
categories = ['Very Weak\n(<0.05)', 'Weak\n(0.05-0.1)', 'Moderate\n(0.1-0.2)', 'Strong\n(≥0.2)']
counts = [very_weak, weak, moderate, strong]
bars = ax3.bar(categories, counts, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'], edgecolor='black')
ax3.set_ylabel('Number of Features'); ax3.set_title('Signal Strength Distribution', fontweight='bold')
for bar, count in zip(bars, counts):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{count}', ha='center')

ax4 = axes[1, 1]
ax4.axis('off')
max_corr = corr_df['Abs_Pearson'].max()
mean_corr = corr_df['Abs_Pearson'].mean()
pct_weak = (corr_df['Abs_Pearson'] < 0.1).sum() / len(corr_df) * 100
summary = f"""
Signal-to-Noise Summary
═══════════════════════

Features: {len(corr_df)}
Max |r|:  {max_corr:.4f}
Mean |r|: {mean_corr:.4f}

Very Weak: {very_weak} ({very_weak/len(corr_df)*100:.0f}%)
Weak:      {weak} ({weak/len(corr_df)*100:.0f}%)
Moderate:  {moderate} ({moderate/len(corr_df)*100:.0f}%)
Strong:    {strong} ({strong/len(corr_df)*100:.0f}%)

⚠️ {pct_weak:.0f}% features have |r| < 0.1

Conclusion: LOW signal-to-noise
ratio explains AUC ≈ 0.50
"""
ax4.text(0.1, 0.9, summary, transform=ax4.transAxes, fontsize=13, verticalalignment='top',
         fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
plt.savefig('signal_to_noise_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n  Max |correlation|: {max_corr:.4f}")
print(f"  Features with |r| < 0.1: {pct_weak:.0f}%")
print("\n Additional plots generated!")